## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [Clinical De-identification (EN)](https://aws.amazon.com/marketplace/pp/prodview-ghf7wfm7o2wva).
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Deidentify Clinical Document (EN)

- **Model**: [clinical_deidentification_docwise_wip_en](https://nlp.johnsnowlabs.com/2024/10/03/clinical_deidentification_docwise_wip_en.html)

- **Model Description**: This pipeline can be used to deidentify PHI information from medical texts. The PHI information will be masked and obfuscated in the resulting text. 
The pipeline can mask and obfuscate `LOCATION`, `CONTACT`, `PROFESSION`, `NAME`, `DATE`, `ID`, `AGE`, `MEDICALRECORD`, `ORGANIZATION`, `HEALTHPLAN`, `DOCTOR`, `USERNAME`,
`LOCATION-OTHER`, `URL`, `DEVICE`, `CITY`, `ZIP`, `STATE`, `PATIENT`, `COUNTRY`, `STREET`, `PHONE`, `HOSPITAL`, `EMAIL`, `IDNUM`, `BIOID`, `FAX`, `LOCATION_OTHER`, `DLN`,
`SSN`, `ACCOUNT`, `PLATE`, `VIN`, `LICENSE`, `IP` entities.

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [ ]:
import json
import os
import boto3
import pandas as pd
import sagemaker as sage
from sagemaker import ModelPackage
from sagemaker import get_execution_role
from IPython.display import display
from urllib.parse import urlparse

In [ ]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [4]:
model_name = "clinical-deidentification-docwise-wip-en"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.xlarge"

## 2. Create a deployable model from the model package.

In [5]:
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)

### Input Format

To use the model, you need to provide input in one of the following supported formats:

#### JSON Format

Provide input as JSON. We support two variations within this format:

1. **Array of Text Documents**: 
   Use an array containing multiple text documents. Each element represents a separate text document.

   ```json
   {
       "text": [
           "Text document 1",
           "Text document 2",
           ...
       ]
   }

    ```

2. **Single Text Document**:
   Provide a single text document as a string.


   ```json
    {
        "text": "Single text document"
    }
   ```

#### JSON Lines (JSONL) Format

Provide input in JSON Lines format, where each line is a JSON object representing a text document.

```
{"text": "Text document 1"}
{"text": "Text document 2"}
```

### Important Parameter

- **masking_policy**: `str`

    Users can select a masking policy to determine how sensitive entities are handled:

    - **masked**: Default policy that masks entities with their type.

      Example: "My name is Mike. I was admitted to the hospital yesterday."  
      -> "My name is `<PATIENT>`. I was admitted to the hospital yesterday."

    - **obfuscated**: Replaces sensitive entities with random values of the same type.

      Example: "My name is Mike. I was admitted to the hospital yesterday."  
      -> "My name is `Barbaraann Share`. I was admitted to the hospital yesterday."

    
You can specify these parameters in the input as follows:

```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ],
    "masking_policy": "masked",
}
```


## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Deploy the SageMaker model to an endpoint

In [ ]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

Once endpoint has been created, you would be able to perform real-time inference.

In [7]:
def invoke_realtime_endpoint(record, content_type="application/json", accept="application/json"):
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=json.dumps(record) if content_type == "application/json" else record,
    )

    response_body = response["Body"].read().decode("utf-8")

    if accept == "application/json":
        return json.loads(response_body)
    elif accept == "application/jsonlines":
        return response_body
    else:
        raise ValueError(f"Unsupported accept type: {accept}")

### Initial Setup

In [ ]:
docs = [
'''Mr. William Garcia, 25 years-old , born in Beijing, was transfered to the The Johns Hopkins Hospital. Phone number: (541) 754-3010. MSW: 100009632582 . for his colonic polyps.  William Garcia wants to know the results from them.  He is not taking hydrochlorothiazide and is curious about his blood pressure. He said he has cut his alcohol back to 6 pack once a week.  William Garcia has cut back his cigarettes to one time per week. MR. # 50712249 P:   Follow up with Dr. Hobbs Spruce in 3 months. Gilbert P. Perez, M.D. Johns Hopkins Hospital 1800 Orleans St. PHONE: (+1) 410-955-5000 ''',

'''Record date : 2022-01-13 , Ethan Hale , M.D . , Name : Ava Davis. EMAIL: davis_a@gmail.com. MR # 7190334  Phone : (555) 555-1212. Date : 01-13-1993 PCP : Oliveira , 29 years-old. A long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities.  Ava Davis was noted to have a large sacral wound; this is in a similar location with his previous laminectomy, and this continues to receive daily care. DD: 01/13/2022 . DV: 01/13/2022. Ethan Hale , M.D. MSW 754443200936. NewYork-Presbyterian Hospital . 525 East 68th Street. (85) 555-1212. ''',

]

sample_text = """Record date : 2013-01-13, David Hale, M.D. is manager,  Name: Elvis Presley. Age: 17. Phone: (9) 7765-5632. MR. # 7194334 Date: 01-13-1993 PCP: Oliveira. Record date: 2012-11-09. Cocke County Baptist Hospital 0295 Keats Street. This 17-yr-old male, presented with chest heaviness that started during a pick-up basketball game. His past medical history was unremarkable. Elvis Presley denied prior cardiac symptoms and suffered no chest trauma during the game. His father had suffered an acute myocardial infarction at age 38. Elvis Presley was a nonsmoker, did not drink alcohol, and denied recreational drug use. Elvis Presley swallowed a tablet of aspirin before coming to the emergency room. His blood pressure was 160/90 mm Hg, and his heart rate was 80 bpm. Physical examination revealed no stigmata of Marfan syndrome. The rest of his physical examination was normal. DD: 01/13/2013 . DV: 01/13/2013, Cocke County Baptist Hospital 0295 Keats Street. PHONE : (+1) 423-625-2200 """

### JSON

#### Example 1: masked (default-policy)

In [9]:
input_json_data = {"text": sample_text}
response_json = invoke_realtime_endpoint(input_json_data, content_type="application/json", accept="application/json")
pd.DataFrame(response_json)

,predictions
0,"Record date : <DATE>, <DOCTOR>, M.D. is <PROFESSION>, Name: <PATIENT>. Age: <AGE>. Phone: (<AGE>) <AGE>5-5632. MR. # <MEDICALRECORD> Date: <DATE> PCP: <DOCTOR>. Record date: <DATE>. <HOSPITAL> <STREET>. This <AGE>-yr-old male, presented with chest heaviness that started during a <PROFESSION>. His past medical history was unremarkable. <PATIENT> denied prior cardiac symptoms and suffered no chest trauma during the game. His father had suffered an acute myocardial infarction at age <AGE>. Elvis <PATIENT> was a nonsmoker, did not drink alcohol, and denied recreational drug use. <PATIENT> swallowed a tablet of aspirin before coming to the emergency room. His blood pressure was 160/90 mm Hg, and his heart rate was 80 bpm. Physical examination revealed no stigmata of Marfan syndrome. The rest of his physical examination was normal. DD: <DATE> . DV: <DATE>, Cocke <LOCATION> <STREET>. PHONE : <PHONE>"


#### Example 2: obfuscated

In [11]:
input_json_data = {"text": sample_text, "masking_policy": "obfuscated"}
response_json = invoke_realtime_endpoint(input_json_data, content_type="application/json", accept="application/json")
pd.DataFrame(response_json)

,predictions
0,"Record date : 2013-02-05, Danley Dusky, M.D. is Energy manager, Name: Thad Files. Age: 16. Phone: (10) 1405-5632. MR. # 3752992 Date: 02-05-1993 PCP: Precious. Record date: 2012-12-02. BANNER GATEWAY MEDICAL CENTER 8026 Floyd Curl Dr. This 16-yr-old male, presented with chest heaviness that started during a Pharmacologist. His past medical history was unremarkable. Thad Files denied prior cardiac symptoms and suffered no chest trauma during the game. His father had suffered an acute myocardial infarction at age 36. Elvis Angeline was a nonsmoker, did not drink alcohol, and denied recreational drug use. Thad Files swallowed a tablet of aspirin before coming to the emergency room. His blood pressure was 160/90 mm Hg, and his heart rate was 80 bpm. Physical examination revealed no stigmata of Marfan syndrome. The rest of his physical examination was normal. DD: 02/05/2013 . DV: 02/05/2013, Cocke 56 45 Main St 8026 Floyd Curl Dr. PHONE : (+1) 643-845-4422"


### JSON Lines

In [13]:
def create_jsonl(records, masking_policy=None):
    json_records = []

    if isinstance(records, str):
        records = [records]

    for text in records:
        record = {"text": text}

        if masking_policy is not None:
            record["masking_policy"] = masking_policy
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)
    return json_lines


#### Example 1: masked (default-policy)

In [14]:
input_jsonl_data = create_jsonl(sample_text, masking_policy="masked")
data = invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": "Record date : <DATE>, <DOCTOR>, M.D. is <PROFESSION>,  Name: <PATIENT>. Age: <AGE>. Phone: (<AGE>) <AGE>5-5632. MR. # <MEDICALRECORD> Date: <DATE> PCP: <DOCTOR>. Record date: <DATE>. <HOSPITAL> <STREET>. This <AGE>-yr-old male, presented with chest heaviness that started during a <PROFESSION>. His past medical history was unremarkable. <PATIENT> denied prior cardiac symptoms and suffered no chest trauma during the game. His father had suffered an acute myocardial infarction at age <AGE>. Elvis <PATIENT> was a nonsmoker, did not drink alcohol, and denied recreational drug use. <PATIENT> swallowed a tablet of aspirin before coming to the emergency room. His blood pressure was 160/90 mm Hg, and his heart rate was 80 bpm. Physical examination revealed no stigmata of Marfan syndrome. The rest of his physical examination was normal. DD: <DATE> . DV: <DATE>, Cocke <LOCATION> <STREET>. PHONE : <PHONE> "}


#### Example 2: obfuscated

In [16]:
input_jsonl_data = create_jsonl(sample_text, masking_policy="obfuscated")
data = invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": "Record date : 2013-02-05, Danley Dusky, M.D. is Energy manager,  Name: Thad Files. Age: 16. Phone: (10) 1405-5632. MR. # 3752992 Date: 02-05-1993 PCP: Precious. Record date: 2012-12-02. BANNER GATEWAY MEDICAL CENTER 8026 Floyd Curl Dr. This 16-yr-old male, presented with chest heaviness that started during a Pharmacologist. His past medical history was unremarkable. Thad Files denied prior cardiac symptoms and suffered no chest trauma during the game. His father had suffered an acute myocardial infarction at age 36. Elvis Angeline was a nonsmoker, did not drink alcohol, and denied recreational drug use. Thad Files swallowed a tablet of aspirin before coming to the emergency room. His blood pressure was 160/90 mm Hg, and his heart rate was 80 bpm. Physical examination revealed no stigmata of Marfan syndrome. The rest of his physical examination was normal. DD: 02/05/2013 . DV: 02/05/2013, Cocke 56 45 Main St 8026 Floyd Curl Dr. PHONE : (+1) 643-845-4422 "}


### B. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [19]:
validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/jsonl/"

def upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/{file_format[1:]}/{file_name}",
        Body=input_data.encode("UTF-8"),
    )

In [20]:
# Create JSON and JSON Lines data
input_json_data = {
    "input1.json": json.dumps({"text": docs, "masking_policy": "masked"}),
    "input2.json": json.dumps({"text": docs, "masking_policy": "obfuscated"}),
}

input_jsonl_data = {
    "input1.jsonl": create_jsonl(docs, masking_policy="masked"),
    "input2.jsonl": create_jsonl(docs, masking_policy="obfuscated"),
}

# Upload JSON and JSON Lines data to S3
for file_name, data in input_json_data.items():
    upload_to_s3(data, file_name)

for file_name, data in input_jsonl_data.items():
    upload_to_s3(data, file_name)


### JSON

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [ ]:
def retrieve_json_output_from_s3(validation_file_name):
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    display(data)

In [23]:
masking_policies = {
    "masked": "input1.json",
    "obfuscated": "input2.json",
}

for policy_name, validation_file_name in masking_policies.items():
    print(f"Masking policy: {policy_name}")
    retrieve_json_output_from_s3(validation_file_name)
    print("\n")

Masking policy: masked


{'predictions': ['Mr. <PATIENT>, <AGE> years-old , born in <CITY>, was transfered to the The <HOSPITAL>. Phone number: <PHONE>. MSW: <PHONE> . for his colonic polyps.  <PATIENT> wants to know the results from them.  He is not taking hydrochlorothiazide and is curious about his blood pressure. He said he has cut his alcohol back to 6 pack once a week.  <PATIENT> has cut back his cigarettes to one time per week. MR. # <IDNUM> P:   Follow up with Dr. <CITY> <CITY> in 3 months. <DOCTOR> P. <DOCTOR>, M.D. <HOSPITAL> <STREET>. PHONE: <PHONE> ',
  'Record date : <DATE> , <DOCTOR> , M.D . , Name : <DOCTOR>. <NAME>: <EMAIL>. MR # <MEDICALRECORD>  Phone : <PHONE>. Date : <DATE> PCP : <DOCTOR> , <AGE> years-old. A long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities.  Ava <PATIEN



Masking policy: obfuscated


{'predictions': ["Mr. Vernal Gold, 31 years-old , born in Port John, was transfered to the The CHILDREN'S HOSPITAL. Phone number: (349) 534-1090. MSW: 122229834504 . for his colonic polyps.  Vernal Gold wants to know the results from them.  He is not taking hydrochlorothiazide and is curious about his blood pressure. He said he has cut his alcohol back to 6 pack once a week.  Vernal Gold has cut back his cigarettes to one time per week. MR. # 52714469 P:   Follow up with Dr. RABENFURT Kimberlyberg in 3 months. Lorilee P. Treasa, M.D. CHILDREN'S HOSPITAL 1007 Goodyear Avenue. PHONE: (+1) 612-955-5222 ",
  'Record date : 2022-02-05 , Johnna Nakai , M.D . , Name : Carlynn Chiles. AUDRA: Guy@hotmail.com. MR # 5970114  Phone : (222) 222-8181. Date : 02-05-1993 PCP : Precious , 26 years-old. A long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outs

### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [ ]:
def retrieve_jsonlines_output_from_s3(validation_file_name):

    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

In [26]:
masking_policies = {
    "masked": "input1.jsonl",
    "obfuscated": "input2.jsonl",
}

for policy_name, validation_file_name in masking_policies.items():
    print(f"Masking policy: {policy_name}")
    retrieve_jsonlines_output_from_s3(validation_file_name)
    print("\n")

Masking policy: masked
{"predictions": "Mr. <PATIENT>, <AGE> years-old , born in <CITY>, was transfered to the The <HOSPITAL>. Phone number: <PHONE>. MSW: <PHONE> . for his colonic polyps.  <PATIENT> wants to know the results from them.  He is not taking hydrochlorothiazide and is curious about his blood pressure. He said he has cut his alcohol back to 6 pack once a week.  <PATIENT> has cut back his cigarettes to one time per week. MR. # <IDNUM> P:   Follow up with Dr. <CITY> <CITY> in 3 months. <DOCTOR> P. <DOCTOR>, M.D. <HOSPITAL> <STREET>. PHONE: <PHONE> "}
{"predictions": "Record date : <DATE> , <DOCTOR> , M.D . , Name : <DOCTOR>. <NAME>: <EMAIL>. MR # <MEDICALRECORD>  Phone : <PHONE>. Date : <DATE> PCP : <DOCTOR> , <AGE> years-old. A long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraple

In [ ]:
model.delete_model()

### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

